<a href="https://colab.research.google.com/github/lauren-safwat/World-University-Rankings-Dashboard/blob/main/World_University_Rankings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [242]:
# !pip3 install dash
# !pip3 install jupyter_dash
# !pip3 install dash_bootstrap_components
# !pip install dash-trich-components

In [243]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import jupyter_dash
from dash import Dash, html, dcc, no_update
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc
import dash_trich_components as dtc

import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

from math import ceil
from itertools import cycle

In [244]:
# import shutil
# shutil.rmtree('/content/World-University-Rankings-Dashboard')

In [245]:
# !git clone https://github.com/lauren-safwat/World-University-Rankings-Dashboard

In [246]:
%cd /content/World-University-Rankings-Dashboard
unis = pd.read_csv('dataset/World_University_Rankings.csv')

/content/World-University-Rankings-Dashboard


In [247]:
app = jupyter_dash.JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP, 'assets/css/style.css'], suppress_callback_exceptions=True)

# **Distribution of universities across the world page**

In [241]:
world = dbc.Container([
    dbc.Row(html.H1('Distribution of Universities Across the Torld')),
    dbc.Row([
        dbc.Col(dcc.Graph(id='map', figure={}), width=9),
        dbc.Col(html.Div(id='table', style={'maxHeight': '450px', 'overflowY': 'scroll'}),width=3)
    ], id='map_table'),

    dbc.Row([
        dbc.Col(dcc.Slider(id='mapSlider',
                  min=2017,
                  max=2022,
                  value=2022,
                  step=None,
                  marks={i: str(i) for i in range(2017, 2023)}
                  ), width=6),
             
        dbc.Col(dcc.Dropdown([str(region) for region in unis['Region'].unique()], placeholder='Select Region', id='map_region'), width=2),
        dbc.Col(dcc.Dropdown(placeholder='Select Country', id='map_country'),width=2),
        dbc.Col(dcc.Checklist([
            {'label': 'Private Universities', 'value': 'Private'},
            {'label': 'Public Universities', 'value': 'Public'},
        ],
        id='checklist'
      ),width=2)
    ],id='controls_row'),
    

    dbc.Row([
        dbc.Col(
            dbc.Row(dcc.Graph(id='bar_uniCount', figure={})),
            width=4
        ),

        dbc.Col(
            dbc.Row(dcc.Graph(id='bar_intCount', figure={})),
            width=4
        ),
        dbc.Col(
            dbc.Row(dcc.Graph(id='pie_type', figure={})),
            width=4
        )
      ]),
    
    ])

SyntaxError: ignored

In [248]:
@app.callback(
    Output('map_country', 'options'),
    Output('map', 'figure'),
    Output("table", "children"),
    Input('mapSlider', 'value'),
    Input('map_region', 'value'),
    Input('map_country', 'value'),
    Input('checklist', 'value')
)


def updateMap(year, region, country,priv):
    year = 'Rank_' + str(year)
    df = unis[unis[year] > 0].sort_values(year, axis=0)
    print
    zoom = 0.1

    if region:
        zoom = 1.3
        df = df[df['Region'] == region]

    if country:
        zoom = 2.5
        df = df[df['Country'] == country]
    if(priv):
      df=df[df['Type'].isin(priv)]

    
    fig = px.scatter_mapbox(df[:100],
                            lon='Longitude',
                            lat='Latitude',
                            color=year,
                            hover_name='University',
                            hover_data={'Latitude':False, 'Longitude':False, 'Country':True,'City':True},
                            zoom=zoom,
                            title = 'Ranking of Universities Around the World',
                            mapbox_style='carto-positron',
                            color_continuous_scale=px.colors.sequential.deep_r,
                            )
    
    fig.update_traces(marker = {'size':10, 'opacity':0.5}, selector={'type': 'scattermapbox'})

    countries = [{'label': str(i), 'value':str(i)} for i in df['Country'].unique()]

    table = dbc.Table.from_dataframe(df[[year, 'University']], striped=True, bordered=True, hover=True, responsive=True)
    
    return countries, fig, table

In [249]:
@app.callback(
    Output('bar_uniCount', 'figure'),
    Input('map_region', 'value'),
    Input('checklist', 'value')
)

def updateBar1(region, priv):

    #countries = [{'label': str(i), 'value':str(i)} for i in unis['Country'].unique()]
    x=unis.groupby(['Region'])['University'].count()
    y=x.keys()
    text1 = 'Region'    
    df=unis
    if(priv):
      df=df[df['Type'].isin(priv)]
      x=df.groupby(['Region'])['University'].count()
      y=x.keys()
    if region:
        df = df[df['Region'] == region]
        #countries = [{'label': str(i), 'value':str(i)} for i in df['Country'].unique()]
        x=df.groupby(['Country'])['University'].count()
        y=x.keys()
        text1 = 'Country'
    sortd = sorted(zip(x,y))
    x = [i[0] for i in sortd]
    y = [i[1] for i in sortd]
        
    fig = go.Figure(go.Bar(
                x=list(x),
                y=list(y),
                orientation='h',
                text=x,
                hovertemplate = "<br>"+text1+": %{y} </br> Count:%{text}<extra></extra>",
                marker={
                'color': x,
                'colorscale': 'bluyl'
                }))
    fig.update_layout(title="University Count",
                      barmode='group',
                      bargap=0.0,
                      bargroupgap=0.0
                     )
    return fig

In [250]:
@app.callback(
    Output('bar_intCount', 'figure'),
    Input('map_region', 'value'),
    Input('checklist', 'value')
)

def updateBar2(region,priv):

    #countries = [{'label': str(i), 'value':str(i)} for i in unis['Country'].unique()]
    x=unis.groupby(['Region'])['International_Students'].sum()
    y=x.keys()
    text1 = 'Region'    
    df=unis
    if(priv):
      df=df[df['Type'].isin(priv)]
      x=df.groupby(['Region'])['International_Students'].sum()
      y=x.keys()

    if region:
        text1 = 'Country'
        df = df[df['Region'] == region]
        #countries = [{'label': str(i), 'value':str(i)} for i in df['Country'].unique()]
        x=df.groupby(['Country'])['International_Students'].sum()
        y=x.keys()
    sortd = sorted(zip(x,y))
    x = [i[0] for i in sortd]
    y = [i[1] for i in sortd]
    
    fig = go.Figure(go.Bar(
                x=list(x),
                y=list(y),
                orientation='h',
                text=x,
                hovertemplate = "<br>"+text1+": %{y} </br> Count:%{text}<extra></extra>",
                marker={
                'color': x,
                'colorscale': 'bluyl'
                }))
    fig.update_layout(title="Number of International Students",
                      barmode='group',
                      bargap=0.0,
                      bargroupgap=0.0)
    return fig

In [251]:
@app.callback(
    # Output('pie_country_type', 'options'),
    Output('pie_type', 'figure'),
    Input('map_region', 'value'),
    Input('map_country', 'value')
)

def updatePie(region,country):
    df=unis
    labels=['Private','Public']  
    text = 'World'      

    if region:
        df = unis[unis['Region'] == region]
        text = region  
    if country :
      df = df[df['Country'] == country]
      text=country
    
    values=[df[df['Type']=='Private'].count()['University'],df[df['Type']=='Public'].count()['University']]
    countries = [{'label': str(i), 'value':str(i)} for i in df['Country'].unique()]

    fig = go.Figure(data=[go.Pie(labels=labels, values=values,
                             text=values,
                             hole=.3,
                             marker = dict(colors = ['rgb(148,212,180)','rgb(4,82,117,255)']),
                             textinfo='label+percent',
                             insidetextorientation='radial',
                             hovertemplate = "<br>Type: %{label} </br> Count:%{value}<extra></extra>",
                             )])
    fig.update_layout(title="Public Vs Private Universites ")
    fig.add_annotation(dict(font=dict(color='black',size=15),
                                        x=0,
                                        y=-0.12,
                                        showarrow=False,
                                        text=text,
                                        textangle=0,
                                        xanchor='left',
                                        xref="paper",
                                        yref="paper"))
    return fig


# **Universities page**

In [252]:
uni = dbc.Container([
      dbc.Row([
          dbc.Col(html.H1('👩🏻‍🎓 Ranking of Top 5 Universities Over the World', id='page2_title'))
      ]),

      dbc.Row([
          dbc.Col(dcc.Dropdown([str(region) for region in unis['Region'].unique()], placeholder='Select Region',multi=True, id='line_region_uni'),width=4),
          dbc.Col(dcc.Dropdown(placeholder='Select Country',multi=True, id='line_country_uni'),width=4),
          dbc.Col(dcc.Dropdown(placeholder='Select University',multi=True, id='line_uni_uni'),width=4)
      ], className='dropDowns'),  

      dbc.Row(
          dbc.Col(dcc.Graph(id='line_uni', figure={}), width=12)
      ),

      dbc.Row([
          dbc.Col(dtc.Carousel([],
              id='carousel',
              slides_to_scroll=1,
              swipe_to_slide=True,
              autoplay=False,
              arrows=True
          ))
      ])
    
])

In [253]:
@app.callback(
    Output('line_uni_uni', 'options'),
    Output('line_country_uni', 'options'),
    Output('line_uni', 'figure'),
    Input('line_region_uni', 'value'),
    Input('line_country_uni', 'value'),
    Input('line_uni_uni', 'value')
)

def updateLine(region,country,university):
  df = unis
  
  palette = cycle(px.colors.cyclical.Phase)
  palette2=cycle(px.colors.cyclical.Phase)
  if region :
    df = unis[unis['Region'].isin(region) ]
  if country :
    df = df[df['Country'].isin(country)]
  if not university:
    university = list(df[df['Rank_2022']>0].iloc[0:5, 0].values)
  fig = go.Figure()
  for uni in university:
    x=np.arange(2017, 2023)
    y=df[df['University']==uni].iloc[:,11:17].values.flatten().tolist()
    x = [x[i] for i in range(len(y)) if y[i]>0]
    y = [y[i] for i in range(len(y)) if y[i]>0]
    
    fig.add_trace(go.Scatter(x=x, y=y, name=uni,
                        line_shape='linear',
                        line_color=next(palette),
                        hovertemplate = "<br>Year : %{x} </br> Rank:%{y}<extra></extra>"))
    fig.add_trace(go.Scatter(x=[x[y.index(np.min(y))]], y=[np.min(y)],name=str(min(y)),
                            mode = 'markers',
                            marker_symbol = 'circle',
                            marker_color=next(palette2),
                            marker_size = 10,
                            hovertemplate = "<br>Year : %{x} </br> Rank:%{y}<extra></extra>",
                            showlegend=False))

  fig.update_layout(
      yaxis=dict(
          autorange='reversed',
      ),
      xaxis=dict(
          dtick=1,
          range=[2016.1, 2022.9]
      ),
    legend=dict(
        
        title_font_family="Times New Roman",
        font=dict(
            family="Courier",
            size=12,
            color="black"
        ),
       
    )
)
  
  countries = [{'label': str(i), 'value':str(i)} for i in df['Country'].unique()]
  universities = [{'label': str(i), 'value':str(i)} for i in df['University'].unique()]
  return universities, countries, fig



In [254]:
@app.callback(
    Output('carousel', 'children'),
    Input('line_uni_uni', 'value')
)

def updateCards(university):
    universities = unis[unis['Rank_2022']>0][0:5]

    if university:
        universities = unis[unis['University'].isin(university)]

    labels=['International Students', 'National Students']

    cards = []

    for i in range(universities.shape[0]):

        fig = go.Figure()
        international_students = universities.iloc[i, 9]
        students = universities.iloc[i, 10]
        values = [international_students, students-international_students]

        fig.add_trace(go.Pie(labels=labels, values=values,
                                text=values,
                                hole=.3,
                                  marker = dict(colors = ['rgb(148,212,180)','rgb(4,82,117,255)']),
                                textinfo='percent',
                                insidetextorientation='radial',
                                hovertemplate = "<br>%{label} </br> Count:%{value}<extra></extra>",)
        )

        fig.update_layout(margin=dict(t=0, b=0, l=0, r=0))

        card_content = [
          html.A([
              dbc.CardHeader([
                 # html.Img(src = universities.iloc[i, 4]),
                  html.H5(universities.iloc[i, 0])
              ], id='card_header')
          ], href=universities.iloc[i, 5], target='_blank'),

          dbc.CardBody([
              html.Ul(),
              dcc.Graph(figure = fig, style={"height": "80%", "width": "100%"})
          ]),
        ]
        cards.append(dbc.Card(card_content, outline=True,id='uni_card'))
    
    return cards



# **Dashboard Layout**

In [255]:
app.layout = html.Div(children=[
     dbc.Navbar(children=[
        dbc.Col([
            html.Img(id='logo', src=app.get_asset_url('images/World2.png')),
            html.H1('World University Rankings', id='title')
        ], width=8),

        dbc.Col(dbc.Tabs(id='tabs', active_tab='tab-1', children=[
            dbc.Tab(label="World Overview", tab_id='tab-1'),
            dbc.Tab(label="Universities", tab_id='tab-2')
        ]))
     ], id='nav', sticky = 'top'),
   
     html.Div(id='content')
], className='dashboard')

In [256]:
@app.callback(
    Output('content', 'children'),
    Input('tabs', 'active_tab'),
)

def display_content(active_tab):
    if active_tab == 'tab-1':
        return world
    return uni

In [257]:
app.run_server()

Dash app running on:


<IPython.core.display.Javascript object>

In [258]:
# from jupyter_dash import JupyterDash
# from dash import Dash, dcc, html, Input, Output, no_update
# import plotly.graph_objects as go
# import pandas as pd

# ## create sample random data
# df = pd.DataFrame({
#     'x': [80,2,10],
#     'y': [20,3,25],
#     'color': ['red','green','blue'],
#     'img_url': [
#         "https://upload.wikimedia.org/wikipedia/commons/0/03/Refugee_care_near_Poland_border_train_station_20220228.jpg",
#         "https://upload.wikimedia.org/wikipedia/commons/4/45/Ukrainian_refugees_from_2022%2C_crossing_into_Poland.jpg",
#         "https://upload.wikimedia.org/wikipedia/commons/b/b2/Warsaw_Central_Station_during_Ukrainian_refugee_crisis_10.jpg"
        
#     ]
# })

# fig = go.Figure(data=[
#     go.Scatter(
#         x=df['x'], 
#         y=df['y'], 
#         mode='markers',
#         marker=dict(color=df['color'])
#     )
# ])


# fig.update_traces(hoverinfo="none", hovertemplate=None)

# app = JupyterDash(_name_)

# server = app.server

# app.layout = html.Div([
#     dcc.Graph(id="graph-basic-2", figure=fig, clear_on_unhover=True),
#     dcc.Tooltip(id="graph-tooltip", direction='bottom'),
# ])


# @app.callback(
#     Output("graph-tooltip", "show"),
#     Output("graph-tooltip", "bbox"),
#     Output("graph-tooltip", "children"),
#     Output("graph-tooltip", "direction"),
    
#     Input("graph-basic-2", "hoverData"),
# )
# def display_hover(hoverData):
#     if hoverData is None:
#         return False, no_update, no_update ,no_update

#     # demo only shows the first point, but other points may also be available
#     pt = hoverData["points"][0]
#     bbox = pt["bbox"]
#     num = pt["pointNumber"]
    
    
    
#     df_row = df.iloc[num]
#     img_src = df_row['img_url']

#     children = [
#         html.Img(
#             src=img_src,
#             style={"width": "250px"},
#         ),
#         html.P("ukrain refugees"),
#     ]
    
#     y = hoverData["points"][0]['y']
#     direction = "bottom" if y > 2 else "top"
    
#     return True, bbox, children , direction

# app.run_server(mode="inline")